In [1]:
import numpy as np
from numpy import exp,max, sqrt
from FERM import Binomial_Option

# Parameters used throughout the exercise as constants
S0 = 100            # Spot price
r = 0.02            # Fixed interest rate
sigma = 0.30        # Volatility
c = 0.01            # Dividend yield
n = 15+1

**1-5**

# Homework III

### 1)
*Compute the price of an American call option with strike $K=110$ and maturity $T=.25$ years*

In [3]:
K = 110             # Strike Price
T = 0.25            # Time to maturity

option1 = Binomial_Option(S0, r, sigma, n, T, c)
price = option1.option_price(K, form="call", style="american")
print("${:.2f}".format(price))

$2.60


### 2)
Compute the price of an American put option with strike $K=110$ and maturity $T=.25$ years.

In [4]:
K = 110             # Strike Price
T = 0.25            # Time to maturity

put1 = Binomial_Option(S0, r, sigma, n, T, c)
price = put1.option_price(K, form="put", style="american")
print("${:.2f}".format(price))

$12.37


### 3), 4)
*Is it ever optimal to exercise the put of Q2? If so, when it's the earliest?*

It is optimal to exercise an american put option at time $t$, whenever 
$$
\max\{C(t), E[C(t+1)]\} = C(t)
$$

In [5]:
# Substract the intrinsic value of the put (what we would recieve, were we to exercise),
# from the risk-neutral expectation at time t.
# If the intrinsic value is greater than the R.N. expected value, then it would be optimal 
# to early exercise.

compare = np.array(put1.intrinsic_value_tree) - np.array(put1.present_val_tree)[1:]
for t, branch in enumerate(compare[::-1]):
    exercise = branch > 0
    print("t:{:>2}".format(t), end=" ")
    for v in exercise:
        print("{:>3}".format(v), end=" ")
    print()

t: 0   0 
t: 1   0   0 
t: 2   0   0   0 
t: 3   0   0   0   0 
t: 4   0   0   0   0   0 
t: 5   0   0   0   0   0   1 
t: 6   0   0   0   0   0   0   1 
t: 7   0   0   0   0   0   0   1   1 
t: 8   0   0   0   0   0   0   1   1   1 
t: 9   0   0   0   0   0   0   0   1   1   1 
t:10   0   0   0   0   0   0   0   1   1   1   1 
t:11   0   0   0   0   0   0   0   1   1   1   1   1 
t:12   0   0   0   0   0   0   0   1   1   1   1   1   1 
t:13   0   0   0   0   0   0   0   1   1   1   1   1   1   1 
t:14   0   0   0   0   0   0   0   1   1   1   1   1   1   1   1 
t:15   0   0   0   0   0   0   0   1   1   1   1   1   1   1   1   1 


At $t=5$, if the option is *deep in the money*, then it is optimal, and the earliest, to exercise the american put.

### 5)
*Do the call and put option prices of Questions 1 and 2 satisfy put-call parity?*

Recall the put-call parity:
$$
    P_E + S = C_E + K
$$

Since *1)* and *2)* are 'american' options, the put-call parity does **not** hold.

### 6)
Compute the fair value of an American call option with strike $K=110$ and maturity $n=10$ periods where the option is written on a **futures contract** that expires after 15 periods. The futures contract is on the same underlying security of the previous questions.

In [6]:
n = 11
K = 110
putF = Binomial_Option(S0*put1.rate**(15), 0, sigma, n, T, c = c)
price = putF.option_price(K, form = "call", style = "american")
print("${:.2f}".format(price))

$2.61


In [12]:
from FERM import Binomial_Option
sigma = 1

putF = Binomial_Option(100, 0, sigma, 6, 6/12)
putF.u = 1.02
putF.d = 0.99
putF.rate = 1.01
putF.tree = putF.asset_tree(futures=True)
print(putF.option_price(102.15))
putF.print_tree("option")

4.27960851848
t  0   4.3
t  1   2.5   5.2
t  2   1.1   3.2   6.1
t  3   0.2   1.5   4.1   7.2
t  4   0.0   0.4   2.1   5.0   8.3
t  5   0.0   0.0   0.5   2.9   6.1   9.4
t  6   0.0   0.0   0.0   0.8   3.9   7.2  10.5


In [13]:
for row in putF.option_price_tree:
    print(row)

[10.466241926400002 7.153999516799999 3.9391760016000035 0.8189061191999798
 0 0 0]
[ 9.36216112  6.08239168  2.89908604  0.54593741  0.          0.        ]
[ 8.26890464  5.0212898   2.11470316  0.36395828  0.        ]
[ 7.18636636  4.05242759  1.53112153  0.24263885]
[ 6.1417201   3.21199224  1.10162731]
[ 5.16514415  2.50853726]
[ 4.27960852]


In [3]:
futures = True
3*2 if futures else 3

6

In [45]:
100 * (1.01)**6

106.15201506010001

In [27]:
putF.tree[0] * 1.02 * (1.01)**(1/12)

array([ 108.36281658])

### 7)
What is the earliest time period in which you might want to exercise the American futures option of Question 6?

[Since:](http://www.math.nyu.edu/~cai/Courses/Derivatives/lecture8.pdf) *Given interest rate r > 0, it is never optimal to exercise an American call between ex-dividends dates or prior to maturity.*, it is better to exercise the call at maturity. i.e. $t=15$

### 8)
Compute the fair value of a chooser option which expires after n=10 periods. At expiration the owner of the chooser gets to choose (at no cost) a European call option or a European put option. The call and put each have strike K=100 and they mature 5 periods later, i.e. at n=15.

In [28]:
chooser = Binomial_Option(S0, r, sigma, 5, T, c = c)
price_call = chooser.option_price(100, form = "call", style = "european")
price_put = chooser.option_price(100, form = "put", style = "european")
chooser_price = (price_call + price_put)
print("${:.2f}".format(chooser_price))

$12.55
